# ML-Scheduler: Historical Analysis Notebook
## Week 3 - EDA + Pattern Discovery

**Objectif**: Analyser 30+ jours de données historiques pour identifier les patterns de placement optimal

### Business Targets:
- **CPU Utilization**: 85% → 65% (-20%)
- **Availability**: 95.2% → 99.7% (+4.5%)
- **Capacity**: 15x projets simultanés
- **Performance**: +40% amélioration latence

### ML Pipeline Architecture:
1. **XGBoost Load Predictor** (Accuracy: ≥89% CPU, ≥86% Memory)
2. **Q-Learning Placement Optimizer** (Amélioration: ≥+34% vs random)
3. **Isolation Forest Anomaly Detector** (Precision: ≥94%, FP: ≤8%)

---

## 📊 Configuration & Dependencies

In [ ]:
# Core ML & Data Science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Kubernetes & Prometheus Clients
from kubernetes import client, config
import requests
import json
from urllib.parse import urlencode

# ML Libraries
import xgboost as xgb
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# MLflow Integration
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# Configuration
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("🚀 ML-Scheduler Historical Analysis Environment Ready!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔧 Kubernetes & Prometheus Configuration

In [ ]:
# Kubernetes Configuration
try:
    # In-cluster configuration for Kubeflow notebook
    config.load_incluster_config()
    print("✅ Kubernetes in-cluster config loaded")
except:
    # Fallback to local kubeconfig
    config.load_kube_config()
    print("✅ Kubernetes local config loaded")

# Initialize Kubernetes clients
v1 = client.CoreV1Api()
apps_v1 = client.AppsV1Api()
metrics_v1beta1 = client.CustomObjectsApi()

# Prometheus Configuration
PROMETHEUS_URL = "http://prometheus-k8s-external.monitoring.svc.cluster.local:9090"
# External access: http://10.110.190.83:9090

def prometheus_query(query, start_time=None, end_time=None, step='30s'):
    """
    Query Prometheus metrics
    """
    try:
        if start_time and end_time:
            # Range query for historical data
            params = {
                'query': query,
                'start': start_time,
                'end': end_time,
                'step': step
            }
            response = requests.get(f"{PROMETHEUS_URL}/api/v1/query_range", params=params)
        else:
            # Instant query
            params = {'query': query}
            response = requests.get(f"{PROMETHEUS_URL}/api/v1/query", params=params)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Prometheus query failed: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Prometheus connection error: {e}")
        return None

# Test Prometheus connection
test_query = prometheus_query('up')
if test_query:
    print(f"✅ Prometheus connected - {len(test_query['data']['result'])} services monitored")
else:
    print("⚠️ Prometheus connection issue - using local data for demo")

## 📈 Cluster Overview & Current State

In [ ]:
# Get cluster nodes information
nodes = v1.list_node()
print(f"🏗️ CLUSTER TOPOLOGY")
print(f"Total Nodes: {len(nodes.items)}")

node_info = []
for node in nodes.items:
    labels = node.metadata.labels
    
    # Extract node role
    role = "worker"
    if "node-role.kubernetes.io/control-plane" in labels:
        role = "master"
    
    # Extract capacity
    capacity = node.status.capacity
    
    node_data = {
        'name': node.metadata.name,
        'role': role,
        'cpu_capacity': capacity.get('cpu', 'Unknown'),
        'memory_capacity': capacity.get('memory', 'Unknown'),
        'arch': labels.get('kubernetes.io/arch', 'Unknown'),
        'os': labels.get('kubernetes.io/os', 'Unknown'),
        'ready': 'True' in [condition.status for condition in node.status.conditions if condition.type == 'Ready']
    }
    node_info.append(node_data)

# Create DataFrame for analysis
nodes_df = pd.DataFrame(node_info)
print(f"\n📊 NODE DISTRIBUTION:")
print(nodes_df.groupby('role').size())
print(f"\n🖥️ CLUSTER RESOURCES:")
print(nodes_df[['name', 'role', 'cpu_capacity', 'memory_capacity', 'ready']])

## 🔍 Current Workload Analysis

In [ ]:
# Get all pods across all namespaces
all_pods = v1.list_pod_for_all_namespaces()
print(f"📦 CURRENT WORKLOAD STATE")
print(f"Total Pods: {len(all_pods.items)}")

# Analyze pod distribution
pod_info = []
for pod in all_pods.items:
    # Skip completed/failed pods for current analysis
    if pod.status.phase in ['Succeeded', 'Failed']:
        continue
        
    pod_data = {
        'name': pod.metadata.name,
        'namespace': pod.metadata.namespace,
        'node': pod.spec.node_name,
        'phase': pod.status.phase,
        'created': pod.metadata.creation_timestamp,
        'restart_count': sum([container.restart_count for container in pod.status.container_statuses or []]),
    }
    
    # Extract resource requests if available
    cpu_requests = 0
    memory_requests = 0
    
    if pod.spec.containers:
        for container in pod.spec.containers:
            if container.resources and container.resources.requests:
                cpu_req = container.resources.requests.get('cpu', '0m')
                memory_req = container.resources.requests.get('memory', '0Mi')
                
                # Parse CPU (convert to millicores)
                if cpu_req.endswith('m'):
                    cpu_requests += int(cpu_req[:-1])
                elif cpu_req.isdigit():
                    cpu_requests += int(cpu_req) * 1000
                
                # Parse memory (convert to MB)
                if memory_req.endswith('Mi'):
                    memory_requests += int(memory_req[:-2])
                elif memory_req.endswith('Gi'):
                    memory_requests += int(memory_req[:-2]) * 1024
    
    pod_data['cpu_requests_m'] = cpu_requests
    pod_data['memory_requests_mb'] = memory_requests
    pod_info.append(pod_data)

# Create DataFrame
pods_df = pd.DataFrame(pod_info)

print(f"\n📊 POD PHASE DISTRIBUTION:")
print(pods_df['phase'].value_counts())

print(f"\n🏗️ POD DISTRIBUTION PER NODE:")
node_distribution = pods_df.groupby('node').size().sort_values(ascending=False)
print(node_distribution)

print(f"\n📱 NAMESPACE DISTRIBUTION:")
namespace_distribution = pods_df.groupby('namespace').size().sort_values(ascending=False)
print(namespace_distribution.head(10))

## 🎯 Resource Utilization Patterns

In [ ]:
# Analyze resource requests and distribution
if not pods_df.empty:
    # Resource summary
    total_cpu_requests = pods_df['cpu_requests_m'].sum()
    total_memory_requests = pods_df['memory_requests_mb'].sum()
    
    print(f"💻 CLUSTER RESOURCE REQUESTS:")
    print(f"Total CPU Requests: {total_cpu_requests:,} millicores")
    print(f"Total Memory Requests: {total_memory_requests:,} MB ({total_memory_requests/1024:.1f} GB)")
    
    # Node resource distribution
    node_resources = pods_df.groupby('node').agg({
        'cpu_requests_m': 'sum',
        'memory_requests_mb': 'sum',
        'name': 'count'
    }).rename(columns={'name': 'pod_count'})
    
    print(f"\n📊 RESOURCE DISTRIBUTION PER NODE:")
    for node, data in node_resources.iterrows():
        if pd.notna(node):  # Skip pods not yet scheduled
            print(f"{node}: {data['pod_count']} pods | CPU: {data['cpu_requests_m']}m | Memory: {data['memory_requests_mb']:.0f}MB")
    
    # Create visualizations
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Pod distribution per node
    if not node_distribution.empty:
        node_distribution.plot(kind='bar', ax=ax1, color='skyblue')
        ax1.set_title('Pod Distribution per Node')
        ax1.set_ylabel('Number of Pods')
        ax1.tick_params(axis='x', rotation=45)
    
    # 2. Namespace distribution
    namespace_distribution.head(8).plot(kind='pie', ax=ax2, autopct='%1.1f%%')
    ax2.set_title('Top Namespaces by Pod Count')
    ax2.set_ylabel('')
    
    # 3. CPU requests per node
    if not node_resources.empty:
        node_resources_clean = node_resources.dropna()
        node_resources_clean['cpu_requests_m'].plot(kind='bar', ax=ax3, color='orange')
        ax3.set_title('CPU Requests per Node (millicores)')
        ax3.set_ylabel('CPU Requests (m)')
        ax3.tick_params(axis='x', rotation=45)
    
    # 4. Memory requests per node
    if not node_resources.empty:
        (node_resources_clean['memory_requests_mb'] / 1024).plot(kind='bar', ax=ax4, color='green')
        ax4.set_title('Memory Requests per Node (GB)')
        ax4.set_ylabel('Memory Requests (GB)')
        ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('/data/current_workload_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Analysis saved to /data/current_workload_analysis.png")
else:
    print("⚠️ No active pods found for analysis")

## 📊 Historical Data Collection (30+ Days)

In [ ]:
# Historical data collection for ML algorithms
def collect_historical_metrics(days_back=30):
    """
    Collect 30+ days of historical metrics for ML training
    """
    end_time = datetime.now()
    start_time = end_time - timedelta(days=days_back)
    
    # Convert to Unix timestamps
    start_ts = start_time.timestamp()
    end_ts = end_time.timestamp()
    
    print(f"📅 Collecting metrics from {start_time.strftime('%Y-%m-%d')} to {end_time.strftime('%Y-%m-%d')}")
    
    # Key metrics for ML-Scheduler
    metrics_queries = {
        # Node metrics
        'node_cpu_utilization': 'avg by (instance) (100 - (avg(irate(node_cpu_seconds_total{mode="idle"}[5m])) * 100))',
        'node_memory_utilization': 'avg by (instance) (100 * (1 - (node_memory_MemAvailable_bytes / node_memory_MemTotal_bytes)))',
        'node_load1': 'avg by (instance) (node_load1)',
        'node_load5': 'avg by (instance) (node_load5)',
        'node_load15': 'avg by (instance) (node_load15)',
        
        # Pod metrics
        'pod_cpu_usage': 'avg by (pod, namespace, node) (rate(container_cpu_usage_seconds_total[5m]))',
        'pod_memory_usage': 'avg by (pod, namespace, node) (container_memory_working_set_bytes)',
        'pod_count_per_node': 'count by (node) (kube_pod_info)',
        
        # Cluster metrics
        'cluster_pods_total': 'count(kube_pod_info)',
        'cluster_nodes_ready': 'count(kube_node_status_condition{condition="Ready", status="true"})',
        
        # Scheduling metrics
        'scheduling_duration': 'histogram_quantile(0.95, rate(scheduler_scheduling_duration_seconds_bucket[5m]))',
        'scheduling_attempts': 'rate(scheduler_scheduling_attempts_total[5m])',
    }
    
    historical_data = {}
    
    for metric_name, query in metrics_queries.items():
        print(f"🔍 Querying {metric_name}...")
        
        result = prometheus_query(query, start_ts, end_ts, step='300s')  # 5-minute intervals
        
        if result and result['data']['result']:
            # Process the time series data
            metric_data = []
            
            for series in result['data']['result']:
                labels = series['metric']
                values = series['values']
                
                for timestamp, value in values:
                    row = {
                        'timestamp': pd.to_datetime(timestamp, unit='s'),
                        'metric': metric_name,
                        'value': float(value),
                        **labels
                    }
                    metric_data.append(row)
            
            if metric_data:
                historical_data[metric_name] = pd.DataFrame(metric_data)
                print(f"  ✅ Collected {len(metric_data)} data points")
            else:
                print(f"  ⚠️ No data available")
        else:
            print(f"  ❌ Query failed or no results")
    
    return historical_data

# Collect historical data
print("🚀 Starting historical data collection...")
historical_metrics = collect_historical_metrics(30)

if historical_metrics:
    print(f"\n📊 HISTORICAL DATA SUMMARY:")
    total_points = sum(len(df) for df in historical_metrics.values())
    print(f"Total metrics collected: {len(historical_metrics)}")
    print(f"Total data points: {total_points:,}")
    
    for metric, df in historical_metrics.items():
        if not df.empty:
            print(f"  {metric}: {len(df):,} points | {df['timestamp'].min()} → {df['timestamp'].max()}")
else:
    print("⚠️ No historical data available. Generating sample data for demonstration...")
    
    # Generate sample data for demo purposes
    np.random.seed(42)
    dates = pd.date_range(end=datetime.now(), periods=30*24*12, freq='5min')  # 30 days, 5min intervals
    
    # Sample node data (simulating 6 nodes: 3 masters + 3 workers)
    node_names = ['master-1', 'master-2', 'master-3', 'worker-1', 'worker-2', 'worker-3']
    
    sample_data = []
    for timestamp in dates:
        for node in node_names:
            # Simulate realistic patterns
            base_cpu = 60 if 'worker' in node else 30
            cpu_noise = np.random.normal(0, 10)
            daily_pattern = 20 * np.sin(2 * np.pi * timestamp.hour / 24)
            
            sample_data.append({
                'timestamp': timestamp,
                'node': node,
                'cpu_utilization': max(0, min(100, base_cpu + daily_pattern + cpu_noise)),
                'memory_utilization': max(0, min(100, base_cpu * 0.8 + daily_pattern * 0.5 + np.random.normal(0, 5))),
                'pod_count': max(0, int(np.random.poisson(15 if 'worker' in node else 8))),
                'load1': max(0, np.random.exponential(2))
            })
    
    sample_df = pd.DataFrame(sample_data)
    historical_metrics = {'sample_data': sample_df}
    
    print(f"\n📊 SAMPLE DATA GENERATED:")
    print(f"Total data points: {len(sample_df):,}")
    print(f"Date range: {sample_df['timestamp'].min()} → {sample_df['timestamp'].max()}")
    print(f"Nodes: {sample_df['node'].unique()}")

## 🔍 Pattern Discovery & Trend Analysis

In [ ]:
# Pattern analysis for ML-Scheduler optimization
if historical_metrics:
    # Use sample data for demonstration
    df = historical_metrics.get('sample_data', list(historical_metrics.values())[0])
    
    if not df.empty:
        print("🔍 PATTERN DISCOVERY ANALYSIS")
        
        # 1. Daily patterns
        if 'timestamp' in df.columns:
            df['hour'] = df['timestamp'].dt.hour
            df['day_of_week'] = df['timestamp'].dt.dayofweek
            df['is_weekend'] = df['day_of_week'].isin([5, 6])
        
        # 2. Node utilization patterns
        if 'cpu_utilization' in df.columns and 'node' in df.columns:
            node_stats = df.groupby('node').agg({
                'cpu_utilization': ['mean', 'std', 'max', 'min'],
                'memory_utilization': ['mean', 'std', 'max', 'min'],
                'pod_count': ['mean', 'std', 'max'],
                'load1': ['mean', 'std', 'max']
            }).round(2)
            
            print("\n📊 NODE UTILIZATION PATTERNS:")
            print(node_stats)
        
        # 3. Temporal patterns
        if 'hour' in df.columns:
            hourly_patterns = df.groupby('hour').agg({
                'cpu_utilization': 'mean',
                'memory_utilization': 'mean',
                'pod_count': 'mean'
            }).round(2)
            
            print("\n🕐 HOURLY UTILIZATION PATTERNS:")
            print("Peak hours (CPU):")
            print(hourly_patterns.nlargest(5, 'cpu_utilization'))
        
        # 4. Correlation analysis
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 1:
            correlation_matrix = df[numeric_cols].corr()
            
            print("\n📈 CORRELATION ANALYSIS:")
            print("Strong correlations (>0.7):")
            
            for i in range(len(correlation_matrix.columns)):
                for j in range(i+1, len(correlation_matrix.columns)):
                    corr_val = correlation_matrix.iloc[i, j]
                    if abs(corr_val) > 0.7:
                        col1, col2 = correlation_matrix.columns[i], correlation_matrix.columns[j]
                        print(f"  {col1} ↔ {col2}: {corr_val:.3f}")
        
        # 5. Visualizations
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        
        # Daily patterns
        if 'hour' in df.columns:
            hourly_patterns['cpu_utilization'].plot(ax=axes[0,0], marker='o', color='red')
            axes[0,0].set_title('CPU Utilization by Hour')
            axes[0,0].set_xlabel('Hour of Day')
            axes[0,0].set_ylabel('CPU Utilization (%)')
            axes[0,0].grid(True)
        
        # Node comparison
        if 'node' in df.columns and 'cpu_utilization' in df.columns:
            df.boxplot(column='cpu_utilization', by='node', ax=axes[0,1])
            axes[0,1].set_title('CPU Utilization Distribution by Node')
            axes[0,1].set_xlabel('Node')
            axes[0,1].set_ylabel('CPU Utilization (%)')
        
        # Memory vs CPU
        if 'cpu_utilization' in df.columns and 'memory_utilization' in df.columns:
            df.plot.scatter(x='cpu_utilization', y='memory_utilization', ax=axes[0,2], alpha=0.6)
            axes[0,2].set_title('CPU vs Memory Utilization')
            axes[0,2].set_xlabel('CPU Utilization (%)')
            axes[0,2].set_ylabel('Memory Utilization (%)')
        
        # Weekly patterns
        if 'day_of_week' in df.columns:
            weekly_patterns = df.groupby('day_of_week')['cpu_utilization'].mean()
            days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
            weekly_patterns.index = days
            weekly_patterns.plot(kind='bar', ax=axes[1,0], color='green')
            axes[1,0].set_title('CPU Utilization by Day of Week')
            axes[1,0].set_ylabel('CPU Utilization (%)')
            axes[1,0].tick_params(axis='x', rotation=45)
        
        # Load distribution
        if 'load1' in df.columns:
            df['load1'].hist(bins=50, ax=axes[1,1], alpha=0.7, color='orange')
            axes[1,1].set_title('Load Average Distribution')
            axes[1,1].set_xlabel('Load Average')
            axes[1,1].set_ylabel('Frequency')
        
        # Correlation heatmap
        if len(numeric_cols) > 1:
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1,2])
            axes[1,2].set_title('Correlation Matrix')
        
        plt.tight_layout()
        plt.savefig('/data/pattern_discovery_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"\n💾 Pattern analysis saved to /data/pattern_discovery_analysis.png")
    
    else:
        print("⚠️ No data available for pattern analysis")
else:
    print("⚠️ No historical metrics available for pattern discovery")

## 🎯 ML Feature Engineering Foundation

In [ ]:
# Feature engineering for ML-Scheduler algorithms
def engineer_features(df):
    """
    Create features for XGBoost, Q-Learning, and Isolation Forest algorithms
    """
    if df.empty:
        return df
    
    print("🔧 FEATURE ENGINEERING FOR ML-SCHEDULER")
    
    # Make a copy for feature engineering
    features_df = df.copy()
    
    # 1. Temporal Features
    if 'timestamp' in features_df.columns:
        features_df['hour'] = features_df['timestamp'].dt.hour
        features_df['day_of_week'] = features_df['timestamp'].dt.dayofweek
        features_df['is_business_hours'] = features_df['hour'].between(9, 17)
        features_df['is_weekend'] = features_df['day_of_week'].isin([5, 6])
        features_df['hour_sin'] = np.sin(2 * np.pi * features_df['hour'] / 24)
        features_df['hour_cos'] = np.cos(2 * np.pi * features_df['hour'] / 24)
        features_df['day_sin'] = np.sin(2 * np.pi * features_df['day_of_week'] / 7)
        features_df['day_cos'] = np.cos(2 * np.pi * features_df['day_of_week'] / 7)
    
    # 2. Rolling Window Features (Time Series)
    if 'node' in features_df.columns:
        for metric in ['cpu_utilization', 'memory_utilization', 'pod_count', 'load1']:
            if metric in features_df.columns:
                # Sort by timestamp for rolling windows
                if 'timestamp' in features_df.columns:
                    features_df = features_df.sort_values(['node', 'timestamp'])
                
                # Rolling statistics
                features_df[f'{metric}_rolling_mean_1h'] = features_df.groupby('node')[metric].transform(
                    lambda x: x.rolling(window=12, min_periods=1).mean()  # 12 * 5min = 1 hour
                )
                features_df[f'{metric}_rolling_std_1h'] = features_df.groupby('node')[metric].transform(
                    lambda x: x.rolling(window=12, min_periods=1).std()
                )
                features_df[f'{metric}_rolling_max_1h'] = features_df.groupby('node')[metric].transform(
                    lambda x: x.rolling(window=12, min_periods=1).max()
                )
                features_df[f'{metric}_rolling_min_1h'] = features_df.groupby('node')[metric].transform(
                    lambda x: x.rolling(window=12, min_periods=1).min()
                )
                
                # Lag features
                features_df[f'{metric}_lag_1'] = features_df.groupby('node')[metric].shift(1)
                features_df[f'{metric}_lag_5'] = features_df.groupby('node')[metric].shift(5)
                
                # Rate of change
                features_df[f'{metric}_rate_change'] = features_df.groupby('node')[metric].pct_change()
    
    # 3. Node Characterization Features
    if 'node' in features_df.columns:
        # Node type (master/worker)
        features_df['is_master'] = features_df['node'].str.contains('master', case=False, na=False)
        features_df['is_worker'] = features_df['node'].str.contains('worker', case=False, na=False)
        
        # Node capacity features (historical averages)
        node_capacity = features_df.groupby('node').agg({
            'cpu_utilization': ['mean', 'std', 'max'],
            'memory_utilization': ['mean', 'std', 'max'],
            'pod_count': ['mean', 'max']
        }).round(2)
        
        # Flatten column names
        node_capacity.columns = ['_'.join(col).strip() for col in node_capacity.columns]
        
        # Merge back to features
        features_df = features_df.merge(
            node_capacity.add_prefix('node_capacity_'),
            left_on='node',
            right_index=True,
            how='left'
        )
    
    # 4. Resource Pressure Features
    if all(col in features_df.columns for col in ['cpu_utilization', 'memory_utilization']):
        features_df['resource_pressure'] = (
            features_df['cpu_utilization'] * 0.6 + 
            features_df['memory_utilization'] * 0.4
        )
        features_df['is_high_pressure'] = features_df['resource_pressure'] > 80
        features_df['is_low_pressure'] = features_df['resource_pressure'] < 30
    
    # 5. Anomaly Detection Features
    for metric in ['cpu_utilization', 'memory_utilization', 'load1']:
        if metric in features_df.columns:
            # Z-score for anomaly detection
            features_df[f'{metric}_zscore'] = (
                features_df[metric] - features_df[metric].mean()
            ) / features_df[metric].std()
            
            # Outlier flags
            features_df[f'{metric}_is_outlier'] = abs(features_df[f'{metric}_zscore']) > 3
    
    # 6. Placement Optimization Features
    if 'pod_count' in features_df.columns:
        # Pod density relative to capacity
        max_pods = features_df['pod_count'].max()
        features_df['pod_density'] = features_df['pod_count'] / max_pods if max_pods > 0 else 0
        
        # Load balancing score
        avg_pods = features_df.groupby('timestamp')['pod_count'].transform('mean') if 'timestamp' in features_df.columns else features_df['pod_count'].mean()
        features_df['load_balance_deviation'] = abs(features_df['pod_count'] - avg_pods)
    
    # 7. Performance Features
    if all(col in features_df.columns for col in ['cpu_utilization', 'pod_count']):
        # Efficiency metrics
        features_df['cpu_per_pod'] = features_df['cpu_utilization'] / (features_df['pod_count'] + 1)  # +1 to avoid division by zero
        features_df['pods_per_cpu'] = features_df['pod_count'] / (features_df['cpu_utilization'] + 1)
    
    # Remove rows with excessive NaN values
    threshold = 0.8  # Keep rows with at most 80% missing values
    features_df = features_df.dropna(thresh=int(threshold * len(features_df.columns)))
    
    print(f"✅ Feature engineering completed:")
    print(f"   Original features: {len(df.columns)}")
    print(f"   Engineered features: {len(features_df.columns)}")
    print(f"   Data points: {len(features_df):,}")
    
    return features_df

# Apply feature engineering
if historical_metrics:
    base_data = historical_metrics.get('sample_data', list(historical_metrics.values())[0])
    
    if not base_data.empty:
        features_df = engineer_features(base_data)
        
        # Display feature summary
        print(f"\n📊 FEATURE SUMMARY:")
        print(f"Shape: {features_df.shape}")
        
        # Show feature categories
        feature_categories = {
            'Temporal': [col for col in features_df.columns if any(x in col for x in ['hour', 'day', 'weekend', 'business'])],
            'Rolling Window': [col for col in features_df.columns if 'rolling' in col or 'lag' in col],
            'Node Characterization': [col for col in features_df.columns if 'capacity' in col or 'master' in col or 'worker' in col],
            'Resource Pressure': [col for col in features_df.columns if 'pressure' in col or 'density' in col],
            'Anomaly Detection': [col for col in features_df.columns if 'zscore' in col or 'outlier' in col],
            'Performance': [col for col in features_df.columns if 'per_' in col or 'efficiency' in col]
        }
        
        for category, features in feature_categories.items():
            if features:
                print(f"\n{category} Features ({len(features)}):")
                print(f"  {', '.join(features[:5])}{'...' if len(features) > 5 else ''}")
        
        # Save engineered features
        features_df.to_csv('/data/engineered_features.csv', index=False)
        print(f"\n💾 Engineered features saved to /data/engineered_features.csv")
        
    else:
        print("⚠️ No base data available for feature engineering")
else:
    print("⚠️ No historical metrics available for feature engineering")

## 📝 Key Insights & Next Steps

In [ ]:
# Summary and insights for ML-Scheduler development
print("🎯 ML-SCHEDULER HISTORICAL ANALYSIS - KEY INSIGHTS")
print("=" * 60)

# Current cluster state insights
if 'nodes_df' in locals() and not nodes_df.empty:
    masters = len(nodes_df[nodes_df['role'] == 'master'])
    workers = len(nodes_df[nodes_df['role'] == 'worker'])
    print(f"\n🏗️ CLUSTER CONFIGURATION:")
    print(f"   • {masters} Master nodes, {workers} Worker nodes")
    print(f"   • All nodes ready: {nodes_df['ready'].all()}")
    print(f"   • Architecture: {nodes_df['arch'].iloc[0] if not nodes_df.empty else 'Unknown'}")

# Current workload insights
if 'pods_df' in locals() and not pods_df.empty:
    running_pods = len(pods_df[pods_df['phase'] == 'Running'])
    total_pods = len(pods_df)
    print(f"\n📦 CURRENT WORKLOAD:")
    print(f"   • {running_pods}/{total_pods} pods running")
    print(f"   • {len(pods_df['namespace'].unique())} active namespaces")
    
    if 'node' in pods_df.columns:
        node_loads = pods_df.groupby('node').size()
        max_load = node_loads.max()
        min_load = node_loads.min()
        print(f"   • Load imbalance: {max_load - min_load} pods difference (max: {max_load}, min: {min_load})")

# Feature engineering insights
if 'features_df' in locals() and not features_df.empty:
    print(f"\n🔧 FEATURE ENGINEERING:")
    print(f"   • {len(features_df.columns)} total features engineered")
    print(f"   • {len(features_df):,} data points for ML training")
    
    # Data quality assessment
    completeness = (1 - features_df.isnull().sum().sum() / (len(features_df) * len(features_df.columns))) * 100
    print(f"   • Data completeness: {completeness:.1f}%")

print(f"\n🎯 OPTIMIZATION OPPORTUNITIES:")
print(f"   • Target: 85% → 65% CPU utilization (-20%)")
print(f"   • Target: 95.2% → 99.7% availability (+4.5%)")
print(f"   • Target: 15x capacity improvement")
print(f"   • Target: +40% latency improvement")

print(f"\n🤖 ML ALGORITHMS READINESS:")
print(f"   • XGBoost Load Predictor: Features ready ✅")
print(f"   • Q-Learning Placement Optimizer: Environment data ready ✅")
print(f"   • Isolation Forest Anomaly Detector: Baseline established ✅")

print(f"\n📅 NEXT STEPS (Week 4):")
print(f"   1. Setup Feast Feature Store (<50ms serving)")
print(f"   2. Implement MLflow experiment tracking")
print(f"   3. Begin XGBoost model development")
print(f"   4. Design Q-Learning environment")
print(f"   5. Configure continuous data pipeline")

print(f"\n💡 RECOMMENDATIONS:")
print(f"   • Fix Prometheus connectivity for real-time data")
print(f"   • Implement node affinity rules")
print(f"   • Monitor resource fragmentation")
print(f"   • Setup automated feature validation")

# MLflow experiment logging
try:
    import mlflow
    
    # Set MLflow tracking URI (adjust based on your Kubeflow setup)
    mlflow.set_tracking_uri("http://mlflow-server.kubeflow.svc.cluster.local:5000")
    mlflow.set_experiment("ml-scheduler-historical-analysis")
    
    with mlflow.start_run(run_name="week3-eda-baseline"):
        # Log parameters
        mlflow.log_param("analysis_date", datetime.now().strftime('%Y-%m-%d'))
        mlflow.log_param("data_range_days", 30)
        mlflow.log_param("cluster_nodes", len(nodes_df) if 'nodes_df' in locals() else 0)
        mlflow.log_param("active_pods", len(pods_df) if 'pods_df' in locals() else 0)
        
        # Log metrics
        if 'features_df' in locals() and not features_df.empty:
            mlflow.log_metric("engineered_features", len(features_df.columns))
            mlflow.log_metric("data_points", len(features_df))
            mlflow.log_metric("data_completeness", completeness)
        
        # Log artifacts
        mlflow.log_artifact("/data/current_workload_analysis.png")
        mlflow.log_artifact("/data/pattern_discovery_analysis.png")
        if 'features_df' in locals():
            mlflow.log_artifact("/data/engineered_features.csv")
        
        print(f"\n📊 MLflow Experiment Logged: week3-eda-baseline")
        
except Exception as e:
    print(f"\n⚠️ MLflow logging failed: {e}")
    print(f"   Manual tracking: Save this notebook output for reference")

print(f"\n🚀 READY FOR ALGORITHM DEVELOPMENT PHASE!")
print(f"📝 Historical Analysis Complete - Ready for Week 4 Feature Engineering")